In [2]:
!pip install -U chromadb openai sentence-transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.2/786.2 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 10.2 MB/s et

In [3]:
from google.colab import drive
drive_path= '/content/drive'
drive.mount(drive_path)

Mounted at /content/drive


In [7]:
import json
secret_file_path = "/content/drive/MyDrive/Secret_Keys/OpenAI_Secret_Key.json"
with open(secret_file_path,"r") as f:
  data= json.load(f)
secret_key = data["open_ai_key"]

In [8]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import openai
import uuid


In [ ]:
openai.api_key = secret_key
PERSIST_DIR = "./scoped_memory"
EMBED_MODEL = SentenceTransformer("all-MiniLM-L6-v2")

client= chromadb.PersistentClient(path = PERSIST_DIR)

In [10]:
class ChromaEmbeddingFunction:
  def __call__(self,input):
    return EMBED_MODEL.encode(input).tolist()

  def name(self):
    return "sentence-transformers:all-MiniLM-L6-v2"

embedding_fn = ChromaEmbeddingFunction()

In [11]:
sessions = {}

def create_session(session_name):
  if session_name in sessions:
    print(f"Session '{session_name}' already exists.")
  else:
    collection = client.get_or_create_collection(
        name=session_name,
        embedding_function= embedding_fn
    )
    sessions[session_name] = collection
    print(f"Session '{session_name}' created successfully.")

In [13]:
def add_memory(session_name, text):
  if session_name not in sessions:
    print(f"Session '{session_name}' does not exists.")
    return []
  memory_id = str(uuid.uuid4())
  sessions[session_name].add(
      documents = [text],
      ids = [memory_id],
      metadatas = [{"session":session_name}]
  )
  print("Memory added.")

In [17]:
def recall_with_context(session_name, query):
  if session_name not in sessions:
    print(f"Session '{session_name}' does not exists.")
    return []
  results = sessions[session_name].query(query_texts = query, n_results = 3 )
  context = "\n".join(results["documents"][0]) if results["documents"] else "No memory found."
  prompt = f"Context:\n{context}\n\nQuestion:\n{query}\n\nAnswer:"

  response= openai.chat.completions.create(
      model = "gpt-4o-mini",
      messages= [{"role": "user", "content": prompt }],
      temperature = 0.2
  )
  print("\nGPT response: \n", response.choices[0].message.content)

In [ ]:
# Main menu loop
def run():
    while True:
        print("\nOptions: 1. Create Session  2. Add Memory  3. Recall  4. Exit")
        choice = input("Choose: ")
        if choice == "1":
            name = input("Enter session name: ").strip()
            create_session(name)
        elif choice == "2":
            name = input("Enter session name: ").strip()
            text = input("Enter memory text: ")
            add_memory(name, text)
        elif choice == "3":
            name = input("Enter session name: ").strip()
            query = input("Enter your question: ")
            recall_with_context(name, query)
        elif choice == "4":
            print("Exiting.")
            break
        else:
            print("Invalid choice. Try again.")
# Run the session agent
run()


Options: 1. Create Session  2. Add Memory  3. Recall  4. Exit

GPT response: 
 The word "memory" appears 0 times in the provided context "FirstSessionValue."

Options: 1. Create Session  2. Add Memory  3. Recall  4. Exit
Memory added.

Options: 1. Create Session  2. Add Memory  3. Recall  4. Exit
Session 'who implemented python?' does not exists.

Options: 1. Create Session  2. Add Memory  3. Recall  4. Exit

GPT response: 
 Python was created by Guido van Rossum, not Vivek.

Options: 1. Create Session  2. Add Memory  3. Recall  4. Exit
